# Testing basic LSTM
### creating mock dataset

In [20]:
# create sample data
# try to learn fruit names letter-wise

words = ['apple', 'banana', 'pear', 'lime', 'cherry', 'orange', 'berry', 'strawberry']

# sequence stop sign
SEQUENCE_STOP_SIGN = '.'

# transform words to letter arrays
words_arrays = [[l for l in w] for w in words]

In [40]:
# create sequences in many-to-many configuration
sequences_x, sequences_y = zip(*[(w, w[1:] + [SEQUENCE_STOP_SIGN]) for w in words_arrays])
sequences_x[0], sequences_y[0]

(['a', 'p', 'p', 'l', 'e'], ['p', 'p', 'l', 'e', '.'])

In [32]:
# one-hot encode letters (and stop sign)
import string
ALPHABET = string.ascii_lowercase + SEQUENCE_STOP_SIGN
ALPHABET_LENGTH = len(ALPHABET)

import numpy as np
def encode_letter(letter):
    res = np.zeros(ALPHABET_LENGTH)
    res[ALPHABET.index(letter)] = 1
    return res

def decode_letter(ohe_vec):
    # for now, one dimention only
    return ALPHABET[ohe_vec.argmax()]

encode_letter('d')

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [39]:
def encode_word(word):
    return np.stack([encode_letter(l) for l in word])
    
def decode_word(ohe_word):
    return ''.join([decode_letter(ohe_letter) for ohe_letter in ohe_word])

encode_word('asd')

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [48]:
# encode dataset
data_x = [encode_word(seq) for seq in sequences_x]
data_y = [encode_word(seq) for seq in sequences_y]

In [70]:
# data generator
import random
def data_gen():
    while True:
        index = random.randint(0, len(data_x) - 1)
        yield (
            np.expand_dims(data_x[index], axis=0), 
            np.expand_dims(data_y[index], axis=0)
        )

### create model

In [59]:
import tensorflow as tf

INPUT_UNITS = ALPHABET_LENGTH
HIDDEN_UNITS = 64
OUTPUT_UNITS = ALPHABET_LENGTH

# (seq_len, input)
INPUT_SHAPE = (None, INPUT_UNITS)

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(HIDDEN_UNITS, input_shape=INPUT_SHAPE, return_sequences=True),
    tf.keras.layers.Dense(OUTPUT_UNITS, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['categorical_accuracy']
)


In [68]:
gen = data_gen()
STEPS_PER_EPOCH = len(data_x)
EPOCHS = 10

model.fit_generator(gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)

Epoch 1/10


ValueError: Error when checking target: expected dense_6 to have 3 dimensions, but got array with shape (10, 27)